# Multi Layer Perceptron to classify 9 dance moves 

##### Dataset :  https://www.kaggle.com/nwjbrandon/human-activity-recognition-dancing

There are 9 dance moves - dab, elbowkick, gun, hair, listen, logout, pointhigh, sidepump, and wipetable. The data is in time series and is in raw form - yaw, pitch roll, gyrox, gyroy, gyroz, accx, accy, and accz.

The files are labelled according to the corresponding subject, dance move, and trial number. Using subject3/listen2.csv as an example, the data in this file belongs to subject 3 who is performing the dance move listen for the second time. 

The data is collected from a sensor on the right hand for 1 min at 25Hz. The subject performs the data collection 3 times for 9 dance moves. The subject starts from a standing still position.

In [459]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split 

%matplotlib inline

In [460]:
dance_moves = ["dab", "elbowkick", "gun", "hair", "listen", "logout", "pointhigh", "sidepump", "wipetable"]
dance_moves

['dab',
 'elbowkick',
 'gun',
 'hair',
 'listen',
 'logout',
 'pointhigh',
 'sidepump',
 'wipetable']

In [461]:
dframe = pd.read_csv("./data/subject1/dab1.csv", index_col=None, header = 0) 
dframe.head()

,Unnamed: 0,yaw,pitch,roll,gx,gy,gz,ax,ay,az
0,0,45.98,6.60,-1.07,-10,-4,-3,-1023,481,181
1,1,0.01,-0.01,0.33,-1,0,0,18,-16,-1421
2,2,0.01,-0.02,0.29,-2,0,0,0,11,-370
3,3,0.03,-0.02,0.21,-2,-1,-1,28,16,-41
4,4,0.06,-0.01,0.15,-1,-1,-1,8,4,58


In [462]:
dframe.drop("Unnamed: 0", axis=1, inplace = True)

In [463]:
dframe.shape

(1651, 9)

### Pre-processing 

In [464]:
sub = 1
num = 1
countRows = 0
frames = []
while sub <=7:
    for move in dance_moves:
        while num <=3:
            location = f"data/subject{sub}/{move}{num}.csv"
            df = pd.read_csv(location, index_col=None) 
            df.drop("Unnamed: 0",axis = 1, inplace = True)
            df.dropna(inplace= True)
            df.reset_index()
            frames.append(df)
            print(df)
            countRows += (df.shape[0])
            print("\n")
            print(f"As of last read at {location}, {countRows} rows of data were cumulated")
            print("\n")
            num+=1
        num = 1
    sub+=1
      

rawdata = pd.concat(frames, axis=0, ignore_index=True)




        yaw  pitch   roll  gx  gy  gz    ax    ay    az
0     45.98   6.60  -1.07 -10  -4  -3 -1023   481   181
1      0.01  -0.01   0.33  -1   0   0    18   -16 -1421
2      0.01  -0.02   0.29  -2   0   0     0    11  -370
3      0.03  -0.02   0.21  -2  -1  -1    28    16   -41
4      0.06  -0.01   0.15  -1  -1  -1     8     4    58
...     ...    ...    ...  ..  ..  ..   ...   ...   ...
1646  -0.94  29.10 -49.06   5  -4  -6 -4330  4635  4134
1647  -1.83  28.69 -47.46  -3  -3  -4 -4154  4629  4042
1648  -2.66  28.10 -46.16  -4  -3   0 -4068  4503  4023
1649  -3.72  27.42 -44.75   2  -3   1 -3971  4381  3917
1650  -4.96  26.68 -43.09   6  -1   1 -3886  4320  3756

[1651 rows x 9 columns]


As of last read at data/subject1/dab1.csv, 1651 rows of data were cumulated


        yaw  pitch   roll  gx  gy  gz    ax    ay    az
0    -26.44  -0.70  -5.77  -3  -3  -2    38   764   -67
1     -0.06  -0.01   0.34   0  -1   1   -47  -208 -1308
2     -0.12  -0.03   0.33   0   0   1   -72  -190  -229

         yaw  pitch   roll  gx  gy  gz    ax    ay    az
0    -111.15  69.20 -64.99  11   0   5  -105  1275   127
1       0.06  -0.04   0.24  -2   0  -1   -24   -37 -1323
2       0.13  -0.04   0.20   0  -1  -2   -37   -42  -237
3       0.21  -0.06   0.13  -2   0  -2   -28   -24    32
4       0.27  -0.11   0.01  -3   0  -2   -51    -6   125
...      ...    ...    ...  ..  ..  ..   ...   ...   ...
1637   26.88  46.84 -28.69  49   5  -2 -5886  2350  2350
1638   25.52  45.54 -26.41 -10   6  -6 -5995  2462  2244
1639   25.84  44.29 -26.20 -23   5 -10 -5791  1720  2528
1640   26.25  43.15 -26.15 -23   2 -10 -5694  1385  2626
1641   26.84  42.13 -26.32 -24   2 -13 -5571  1102  2503

[1642 rows x 9 columns]


As of last read at data/subject3/dab1.csv, 89466 rows of data were cumulated


        yaw  pitch   roll  gx  gy  gz    ax    ay    az
0      0.05   0.18   0.69   3  -2  -1   -21   -39 -1322
1      0.08   0.24   0.85   2  -1  -1   -31   -39  -255
2      0.09   0.24   0.97   1  -1  -1   -9

         yaw  pitch   roll  gx  gy  gz    ax    ay    az
0    -103.60   1.36   3.05  -8   0   0  -110   423   120
1      -0.08  -0.01   0.34  -2   0   1   -11   -94 -1458
2      -0.12  -0.06   0.27  -2   0   0    -2   -49  -440
3      -0.09  -0.10   0.14  -3   0  -1    -8   -38  -200
4      -0.07  -0.08   0.09   0  -1  -1    24   -85  -107
...      ...    ...    ...  ..  ..  ..   ...   ...   ...
1721   57.98  42.56 -17.11 -59  15   1 -6860  3124  2132
1722   58.11  40.92 -19.71 -66   4   0 -6290  3200  1951
1723   57.80  39.59 -21.65 -32   0  13 -5662  3200  2035
1724   56.67  38.32 -22.28  -4  -4  25 -4762  3338  1879
1725   55.39  37.41 -22.10   3 -17  23 -5170  3573  1898

[1726 rows x 9 columns]


As of last read at data/subject5/elbowkick3.csv, 183822 rows of data were cumulated


         yaw  pitch   roll  gx  gy  gz    ax    ay    az
0     109.70   1.61   1.48  -3  -1   0   -72  -347  -655
1       0.00  -0.03   0.53   0   0   0    12  -118 -1536
2      -0.01  -0.04   0.54   0  

         yaw  pitch   roll   gx  gy  gz    ax    ay    az
0      -0.02  -0.01   0.43   -1  -1   1     1   -72 -5138
1      -0.09   0.01   0.38   -1  -1   1    24   -51 -1475
2      -0.17   0.01   0.29   -3   0   1    48    -4  -432
3      -0.25   0.01   0.14   -3  -1   1     8    12   -83
4      -0.29  -0.01   0.04   -2   0   0   -12    22   -11
...      ...    ...    ...  ...  ..  ..   ...   ...   ...
1585 -140.90 -19.88  59.37  119 -30  37 -8386 -1209  7006
1586 -142.96 -18.97  63.27   87 -13  30 -8803  -361  7726
1587 -144.63 -18.84  66.85   94 -11  30 -8721  -176  8079
1588 -146.18 -18.82  69.68   54   4  18 -8199   778  7751
1589 -145.82 -19.90  70.27   13  14   4 -7892  1013  7337

[1590 rows x 9 columns]


As of last read at data/subject7/hair1.csv, 282852 rows of data were cumulated


        yaw  pitch   roll   gx  gy   gz    ax    ay    az
0      0.01  -0.01   0.42   -1  -1   -1    -1   -86 -5139
1      0.01   0.01   0.43    0  -1    0     8   -74 -1496
2      0.04   0.04   0

In [465]:
rawdata.tail()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az
310191,-78.17,-18.74,-13.66,-38,-58,-98,1386,-2578,2648
310192,-73.57,-13.30,-17.93,-46,-79,-81,235,-4201,3158
310193,-70.85,-6.83,-21.74,-29,-116,-47,-1613,-6574,4646
310194,-71.99,-1.02,-24.06,-25,-110,7,-3173,-7435,6233
310195,-77.04,1.94,-25.96,-8,-105,80,-5552,-7294,8005


In [466]:
rawdata.head()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az
0,45.98,6.60,-1.07,-10,-4,-3,-1023,481,181
1,0.01,-0.01,0.33,-1,0,0,18,-16,-1421
2,0.01,-0.02,0.29,-2,0,0,0,11,-370
3,0.03,-0.02,0.21,-2,-1,-1,28,16,-41
4,0.06,-0.01,0.15,-1,-1,-1,8,4,58


In [467]:
rawdata.describe()

,yaw,pitch,roll,gx,gy,gz,ax,ay,az
count,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000,310196.000000
mean,-46.726043,-9.851313,-15.575414,1.497850,-3.546132,-1.551210,-3860.604921,-625.942362,4343.535723
std,71.660987,34.015990,31.106017,62.234371,69.444694,115.890831,3810.411576,3154.165889,2978.299761
min,-180.000000,-179.950000,-176.410000,-693.000000,-401.000000,-605.000000,-20182.000000,-19924.000000,-12047.000000
25%,-98.200000,-33.790000,-35.350000,-28.000000,-34.000000,-48.000000,-6250.000000,-2186.000000,2155.000000
50%,-48.535000,-16.630000,-15.020000,1.000000,-2.000000,1.000000,-4315.000000,-298.000000,4372.000000
75%,-0.320000,19.400000,1.270000,32.000000,23.000000,49.000000,-1964.000000,1216.000000,6608.000000
max,180.000000,179.950000,177.980000,538.000000,486.000000,650.000000,24531.000000,672320.000000,13315.000000


In [468]:
type(rawdata.shape)

tuple

In [469]:
"yaw" in rawdata["yaw"]

False

In [470]:
rawdata["yaw"].dtype

dtype('float64')

In [471]:
rawdata.columns

Index(['yaw', 'pitch', 'roll', 'gx', 'gy', 'gz', 'ax', 'ay', 'az'], dtype='object')

In [472]:
rawdata["gx"].dtype

dtype('int64')

In [473]:
for col in rawdata.columns:
    print(f"{col}: {rawdata[col].dtype}")

yaw: float64
pitch: float64
roll: float64
gx: int64
gy: int64
gz: int64
ax: int64
ay: int64
az: int64
